In [2]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [89]:
example = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\SLAT\memory_disorder_SLAT\regex_dataset.xlsx", sheet_name = "Sheet1")

In [90]:
example

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6708893,1.004451e+10,100100942.0,3.115772e+09,Progress Notes,1000021019,955432079,2016-04-28,----------------------------------------------...,"""Visit Other Routine general medical examinati...","[""memory change""]"
1,Z7766129,1.002544e+10,100240713.0,3.116814e+09,H&P,1000431290,954962537,2016-04-28,----------------------------------------------...,"""\u2022 Doxepin Other (See Comments) vivid dre...","[""memory problem""]"
2,Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""accident moving next year due to extra bedroo...","[""dementia""]"
3,Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""techniques discuss again correlations long te...","[""dementia""]"
4,Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""benzo rxs 4/98, 9/97, 5/04 Family history: -m...","[""dementia""]"
...,...,...,...,...,...,...,...,...,...,...,...
23711,Z9833197,7.711140e+05,102405601.0,3.115574e+09,ED Provider Notes,978113347,934708722,2016-04-14,----------------------------------------------...,"""and Coronary artery disease. who presents wit...","[""mild dementia""]"
23712,Z10910426,2.263407e+07,104760040.0,3.110203e+09,Progress Notes,978789346,934607317,2016-04-14,----------------------------------------------...,"""or recreational drug use. Family History: Mot...","[""Alzheimer's disease""]"
23713,Z8133252,4.131140e+05,106467899.0,3.076173e+09,Progress Notes,993123218,948167998,2016-04-25,----------------------------------------------...,"""Family History Problem Relation Age of Onset ...","[""Alzheimer's disease"", ""Alzheimer's disease""]"
23714,Z8133252,4.131140e+05,106467899.0,3.076173e+09,Progress Notes,993123218,948167998,2016-04-25,----------------------------------------------...,"""Age of Onset \u2022 Rheumatoid arthritis Moth...","[""Alzheimer's disease"", ""Alzheimer's disease""]"


In [91]:
example.columns

Index(['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match'],
      dtype='object')

# Padded Matches

In [92]:
# df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Padded_Matches\apoe_1_padded_matches.csv")
# df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\Padded_Matches\apoe_2_padded_matches.csv")
# df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\Padded_Matches\apoe_3_padded_matches.csv")
# df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\Padded_Matches\apoe_4_padded_matches.csv")
# df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\Padded_Matches\apoe_5_padded_matches.csv")

In [93]:
# len(df1), len(df2), len(df3), len(df4), len(df5)

(50577, 45719, 46649, 48325, 48359)

In [94]:
frames = [df1, df2,df3, df4, df5]
df = pd.concat(frames)

In [95]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC', 'buckets',
       'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location', 'regex_sent',
       'regex_match', 'match_count', 'pruned_regex_location',
       'merged_row_location', 'merged_regex_sent', 'merged_regex_match',
       'sequence_level_regex_location', 'sequence_level_regex_match',
       'char_count', 'padded_merged_row_location', 'padded_merged_regex_sent',
       'padded_char_count', 'note_char_count'],
      dtype='object')

In [96]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'EncounterTypeDSC'])
df.head()

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,...,merged_row_location,merged_regex_sent,merged_regex_match,sequence_level_regex_location,sequence_level_regex_match,char_count,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,471433064,Progress Notes,1,4.532520e+08,1.0,...,"[(5750, 5958)]",------- ncer. She passed secondary to this. He...,[' cerebral'],"[(108, 116)]",['cerebral'],213,"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,NaN
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,513142600,Progress Notes,1,4.924833e+08,1.0,...,"[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"[(108, 114)]",['memory'],210,"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,NaN
2,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,2370049295,Progress Notes,1,2.338373e+09,1.0,...,"[(8641, 8849)]","------- ractures, obesity, insulin resistance,...",[' dementia'],"[(108, 116)]",['dementia'],208,"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,NaN
3,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,4959788710,Progress Notes,1,4.951007e+09,1.0,...,"[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(321, 330), (549, 558), (539, 545), (108, 112...","['cognition', 'memory', 'moca']",764,"[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,NaN
4,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,4570521668,Progress Notes,1,4.558686e+09,1.0,...,"[(5367, 5576), (5593, 5812)]",------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(108, 117), (336, 345), (326, 332)]","['cognition', 'memory']",441,"[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,NaN


## Computing Matches For Padded Sequences

In [97]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [98]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[re.compile('\\bdementia\\b', re.IGNORECASE), re.compile('\\bcognition\\b', re.IGNORECASE), re.compile('\\bmemory\\b', re.IGNORECASE), re.compile('\\bMMSE\\b', re.IGNORECASE), re.compile('\\bMOCA\\b', re.IGNORECASE), re.compile('\\bAlzheimer\\b', re.IGNORECASE), re.compile('\\bcognitive\\s*impairment\\b', re.IGNORECASE), re.compile('(?-i:)\\bMCI\\b'), re.compile('\\bcerebellar\\b', re.IGNORECASE), re.compile('\\bNeurocognitive\\b', re.IGNORECASE), re.compile('\\blewy\\b', re.IGNORECASE), re.compile("\\bpick's\\b", re.IGNORECASE), re.compile('\\bCorticobasal\\b', re.IGNORECASE), re.compile('\\bcerebral\\b', re.IGNORECASE), re.compile('\\bcerebrovascular\\b', re.IGNORECASE), re.compile('\\bamnesia\\b', re.IGNORECASE), re.compile('(?-i:)\\bAD\\b'), re.compile('(?-i:)\\bLBD\\b')]


In [99]:
l = []
for note in tqdm(df["padded_merged_regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append((curr))

df["padded_regex_match"] = l

100%|████████████████████████████████████████████████████████████████████████| 239629/239629 [02:28<00:00, 1616.42it/s]


In [100]:
df

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,...,merged_regex_sent,merged_regex_match,sequence_level_regex_location,sequence_level_regex_match,char_count,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,padded_regex_match
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,471433064,Progress Notes,1,4.532520e+08,1.0,...,------- ncer. She passed secondary to this. He...,[' cerebral'],"[(108, 116)]",['cerebral'],213,"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,NaN,[cerebral]
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,513142600,Progress Notes,1,4.924833e+08,1.0,...,------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"[(108, 114)]",['memory'],210,"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,NaN,[memory]
2,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,2370049295,Progress Notes,1,2.338373e+09,1.0,...,"------- ractures, obesity, insulin resistance,...",[' dementia'],"[(108, 116)]",['dementia'],208,"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,NaN,[dementia]
3,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,4959788710,Progress Notes,1,4.951007e+09,1.0,...,------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(321, 330), (549, 558), (539, 545), (108, 112...","['cognition', 'memory', 'moca']",764,"[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,NaN,"[cognition, memory, moca]"
4,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,4570521668,Progress Notes,1,4.558686e+09,1.0,...,------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(108, 117), (336, 345), (326, 332)]","['cognition', 'memory']",441,"[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,NaN,"[cognition, memory]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48354,113722742,10127721883,Z16337903,3.215952e+09,2018-09-05 00:00:00.0000000,2123364929,Progress Notes,5,2.085143e+09,1.0,...,------- INDINGS: The brain parenchyma demonstr...,"[' cerebral', ' cerebellar']","[(108, 118), (544, 554), (327, 335), (579, 587)]","['cerebellar', 'cerebral']",682,"[(10320, 10589), (10838, 11046), (11051, 11353)]",------- pulse sequence(s). MRA of the neck w...,806,17293.0,"[cerebellar, cerebral]"
48355,113722742,10127721883,Z16337903,3.217899e+09,2018-10-03 00:00:00.0000000,2161398176,Progress Notes,5,2.124284e+09,1.0,...,------- ROS performed and are negative. The pa...,"[' cognition', ' memory']","[(108, 117), (326, 332)]","['cognition', 'memory']",423,"[(3531, 3928), (10689, 11083)]",------- ONITORING LT4: TSH Date Value Ref Ran...,809,12288.0,"[cognition, memory]"
48356,113722742,10127721883,Z16337903,3.223494e+09,2018-10-31 00:00:00.0000000,2198013886,Progress Notes,5,2.161922e+09,1.0,...,"------- (R) gait status, as well as getting u...",[' cognition'],"[(108, 117)]",['cognition'],215,"[(1690, 2483)]",e patient felt much more comfortable with and ...,793,4006.0,[cognition]
48357,113722742,10127721883,Z16337903,3.329813e+09,2020-11-04 00:00:00.0000000,4806472815,Progress Notes,5,4.806423e+09,1.0,...,------- her lesions. Normal turgor. [Psych]: A...,[' cognition'],"[(108, 117)]",['cognition'],190,"[(3071, 3871)]","y Social History Narrative Divorced, 1 daughte...",800,3872.0,[cognition]


In [101]:
df.columns

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'buckets', 'NoteCSNID', 'LineNBR',
       'NoteTXT', 'regex_location', 'regex_sent', 'regex_match', 'match_count',
       'pruned_regex_location', 'merged_row_location', 'merged_regex_sent',
       'merged_regex_match', 'sequence_level_regex_location',
       'sequence_level_regex_match', 'char_count',
       'padded_merged_row_location', 'padded_merged_regex_sent',
       'padded_char_count', 'note_char_count', 'padded_regex_match'],
      dtype='object')

## Dropping Rest of Columns

In [102]:
example.columns

Index(['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match'],
      dtype='object')

In [103]:
df = df.drop(columns = ['buckets', 'LineNBR', 'regex_location', 'regex_sent', 'regex_match', 'match_count','pruned_regex_location', 'merged_row_location', 'merged_regex_sent','merged_regex_match', 'sequence_level_regex_location','sequence_level_regex_match', 'char_count','padded_merged_row_location', 'padded_char_count',])

In [104]:
df = df.drop(columns = ['note_char_count'])

In [105]:
df.columns

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT',
       'padded_merged_regex_sent', 'padded_regex_match'],
      dtype='object')

In [106]:
df.columns = ['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT', 'regex_sent', 'regex_match']

In [108]:
df

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,NoteCSNID,NoteTXT,regex_sent,regex_match
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,471433064,Progress Notes,4.532520e+08,----------------------------------------------...,ked at HMS in the department of psychobiology ...,[cerebral]
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,513142600,Progress Notes,4.924833e+08,----------------------------------------------...,or lymphadenopathy. Chest: Clear to auscultati...,[memory]
2,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,2370049295,Progress Notes,2.338373e+09,----------------------------------------------...,ession alone in the meta-analysis. We discusse...,[dementia]
3,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,4959788710,Progress Notes,4.951007e+09,----------------------------------------------...,------- ing tylenol really accept for this we...,"[cognition, memory, moca]"
4,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,4570521668,Progress Notes,4.558686e+09,----------------------------------------------...,------- in is related to cancer. Reassuringly...,"[cognition, memory]"
...,...,...,...,...,...,...,...,...,...,...,...
48354,113722742,10127721883,Z16337903,3.215952e+09,2018-09-05 00:00:00.0000000,2123364929,Progress Notes,2.085143e+09,----------------------------------------------...,------- pulse sequence(s). MRA of the neck w...,"[cerebellar, cerebral]"
48355,113722742,10127721883,Z16337903,3.217899e+09,2018-10-03 00:00:00.0000000,2161398176,Progress Notes,2.124284e+09,----------------------------------------------...,------- ONITORING LT4: TSH Date Value Ref Ran...,"[cognition, memory]"
48356,113722742,10127721883,Z16337903,3.223494e+09,2018-10-31 00:00:00.0000000,2198013886,Progress Notes,2.161922e+09,----------------------------------------------...,e patient felt much more comfortable with and ...,[cognition]
48357,113722742,10127721883,Z16337903,3.329813e+09,2020-11-04 00:00:00.0000000,4806472815,Progress Notes,4.806423e+09,----------------------------------------------...,"y Social History Narrative Divorced, 1 daughte...",[cognition]


In [52]:
df.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", index = False)

In [67]:
df = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", sheet_name = "Sheet1", index_col = 0) 

In [28]:
df['regex_sent'] = df['regex_sent'].apply(json.dumps)
df['regex_match'] = df['regex_match'].apply(json.dumps)

In [69]:
df.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", index = False)

In [3]:
df = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", sheet_name = 'Sheet1', index_col = 0)

In [39]:
df

,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,10040002098,100000272,3.076070e+09,Progress Notes,471433064,4.532520e+08,2015-06-17 00:00:00.0000000,----------------------------------------------...,"""ked at HMS in the department of psychobiology...","""['cerebral']"""
1,10040002098,100000272,3.076214e+09,Progress Notes,513142600,4.924833e+08,2015-07-16 00:00:00.0000000,----------------------------------------------...,"""or lymphadenopathy. Chest: Clear to auscultat...","""['memory']"""
2,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14 00:00:00.0000000,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...","""['dementia']"""
3,10040002098,100000272,3.322836e+09,Progress Notes,4959788710,4.951007e+09,2020-12-04 00:00:00.0000000,----------------------------------------------...,""" ------- ing tylenol really accept for this w...","""['cognition', 'memory', 'moca']"""
4,10040002098,100000272,3.325561e+09,Progress Notes,4570521668,4.558686e+09,2020-09-18 00:00:00.0000000,----------------------------------------------...,""" ------- in is related to cancer. Reassuringl...","""['cognition', 'memory']"""
...,...,...,...,...,...,...,...,...,...,...
48354,10127721883,113722742,3.215952e+09,Progress Notes,2123364929,2.085143e+09,2018-09-05 00:00:00.0000000,----------------------------------------------...,""" ------- pulse sequence(s). MRA of the neck ...","""['cerebellar', 'cerebral']"""
48355,10127721883,113722742,3.217899e+09,Progress Notes,2161398176,2.124284e+09,2018-10-03 00:00:00.0000000,----------------------------------------------...,""" ------- ONITORING LT4: TSH Date Value Ref Ra...","""['cognition', 'memory']"""
48356,10127721883,113722742,3.223494e+09,Progress Notes,2198013886,2.161922e+09,2018-10-31 00:00:00.0000000,----------------------------------------------...,"""e patient felt much more comfortable with and...","""['cognition']"""
48357,10127721883,113722742,3.329813e+09,Progress Notes,4806472815,4.806423e+09,2020-11-04 00:00:00.0000000,----------------------------------------------...,"""y Social History Narrative Divorced, 1 daught...","""['cognition']"""


In [109]:
example.columns

Index(['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match'],
      dtype='object')

In [110]:
df = df[['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match']]

In [111]:
df

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6352398,10040002098,100000272,3.076070e+09,Progress Notes,471433064,4.532520e+08,2015-06-17 00:00:00.0000000,----------------------------------------------...,ked at HMS in the department of psychobiology ...,[cerebral]
1,Z6352398,10040002098,100000272,3.076214e+09,Progress Notes,513142600,4.924833e+08,2015-07-16 00:00:00.0000000,----------------------------------------------...,or lymphadenopathy. Chest: Clear to auscultati...,[memory]
2,Z6352398,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14 00:00:00.0000000,----------------------------------------------...,ession alone in the meta-analysis. We discusse...,[dementia]
3,Z6352398,10040002098,100000272,3.322836e+09,Progress Notes,4959788710,4.951007e+09,2020-12-04 00:00:00.0000000,----------------------------------------------...,------- ing tylenol really accept for this we...,"[cognition, memory, moca]"
4,Z6352398,10040002098,100000272,3.325561e+09,Progress Notes,4570521668,4.558686e+09,2020-09-18 00:00:00.0000000,----------------------------------------------...,------- in is related to cancer. Reassuringly...,"[cognition, memory]"
...,...,...,...,...,...,...,...,...,...,...,...
48354,Z16337903,10127721883,113722742,3.215952e+09,Progress Notes,2123364929,2.085143e+09,2018-09-05 00:00:00.0000000,----------------------------------------------...,------- pulse sequence(s). MRA of the neck w...,"[cerebellar, cerebral]"
48355,Z16337903,10127721883,113722742,3.217899e+09,Progress Notes,2161398176,2.124284e+09,2018-10-03 00:00:00.0000000,----------------------------------------------...,------- ONITORING LT4: TSH Date Value Ref Ran...,"[cognition, memory]"
48356,Z16337903,10127721883,113722742,3.223494e+09,Progress Notes,2198013886,2.161922e+09,2018-10-31 00:00:00.0000000,----------------------------------------------...,e patient felt much more comfortable with and ...,[cognition]
48357,Z16337903,10127721883,113722742,3.329813e+09,Progress Notes,4806472815,4.806423e+09,2020-11-04 00:00:00.0000000,----------------------------------------------...,"y Social History Narrative Divorced, 1 daughte...",[cognition]


# Not-Padded Matches

In [134]:
# df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Padded_Matches\apoe_1_not_padded_matches.csv")
# df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\Padded_Matches\apoe_2_not_padded_matches.csv")
# df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\Padded_Matches\apoe_3_not_padded_matches.csv")
# df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\Padded_Matches\apoe_4_not_padded_matches.csv")
# df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\Padded_Matches\apoe_5_not_padded_matches.csv")

In [197]:
#frames = [df1, df2,df3, df4, df5]
df_not_padded = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_sample_7_19.csv")

In [198]:
example.columns

Index(['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match'],
      dtype='object')

In [199]:
df_not_padded.columns, len(df_not_padded)

(Index(['Unnamed: 0', 'index', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
        'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1',
        'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
        'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC', 'buckets',
        'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location', 'regex_sent',
        'regex_match', 'match_count', 'pruned_regex_location',
        'merged_row_location', 'merged_regex_sent', 'merged_regex_match',
        'padded_merged_row_location', 'padded_merged_regex_sent',
        'padded_char_count', 'note_char_count', 'PCPID',
        'EncounterEpicProviderID', 'pcp_flag', 'SpecialtyDSC',
        'padded_regex_match'],
       dtype='object'),
 5009)

In [200]:
df_not_padded = df_not_padded.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'EncounterTypeDSC'])

,index,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,24365,30123,128460.0,100129465,5114654,Z6923710,3.121598e+09,2016-06-20 00:00:00.0000000,1096403210,Progress Notes,...,[' memory'],"(3296, 4092)","neuralgia, didn't tolerate oxcarbazepin, bacl...",796,4295.0,1009470.0,1007107.0,0,NaN,['memory']
1,47205,58614,255087.0,100335779,10020477146,Z8490045,3.224430e+09,2019-01-16 00:00:00.0000000,2294910337,Progress Notes,...,[' cognition'],"(3172, 3971)",nicteric. [Respiratory]: (+) Slight crackles a...,799,4597.0,1000899.0,1000899.0,1,NaN,['cognition']
2,9334,11514,48761.0,100053905,10041789552,Z6458881,3.176475e+09,2017-10-26 00:00:00.0000000,1736629435,Administrative,...,[' memory'],"(304, 1104)",e. Patient requires physical assistance for pu...,800,1105.0,1000364.0,NaN,0,NaN,['memory']
3,12033,14809,62634.0,100067895,62090,Z6522871,3.243889e+09,2019-04-15 00:00:00.0000000,2415533763,Progress Notes,...,[' dementia'],"(6782, 7580)",Emotionally abused: Not on file Physically ab...,798,10968.0,1009687.0,1672366.0,0,NaN,['dementia']
4,40298,49933,215342.0,100249188,10052853628,Z7830185,3.143184e+09,2017-04-24 00:00:00.0000000,1489471706,MR AVS Snapshot,...,[' memory'],"[(3762, 4152), (5217, 5610)]",------- mouth nightly. gabapentin (NEURONTIN...,801,12097.0,1007039.0,1007039.0,1,NaN,['memory']


In [202]:
df_not_padded = df_not_padded.drop(columns = ['Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1'])

In [205]:
df_not_padded.head()

,index,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,24365,100129465,5114654,Z6923710,3.121598e+09,2016-06-20 00:00:00.0000000,1096403210,Progress Notes,1,1.045850e+09,...,[' memory'],"(3296, 4092)","neuralgia, didn't tolerate oxcarbazepin, bacl...",796,4295.0,1009470.0,1007107.0,0,NaN,['memory']
1,47205,100335779,10020477146,Z8490045,3.224430e+09,2019-01-16 00:00:00.0000000,2294910337,Progress Notes,1,2.263433e+09,...,[' cognition'],"(3172, 3971)",nicteric. [Respiratory]: (+) Slight crackles a...,799,4597.0,1000899.0,1000899.0,1,NaN,['cognition']
2,9334,100053905,10041789552,Z6458881,3.176475e+09,2017-10-26 00:00:00.0000000,1736629435,Administrative,1,1.687146e+09,...,[' memory'],"(304, 1104)",e. Patient requires physical assistance for pu...,800,1105.0,1000364.0,NaN,0,NaN,['memory']
3,12033,100067895,62090,Z6522871,3.243889e+09,2019-04-15 00:00:00.0000000,2415533763,Progress Notes,1,2.385243e+09,...,[' dementia'],"(6782, 7580)",Emotionally abused: Not on file Physically ab...,798,10968.0,1009687.0,1672366.0,0,NaN,['dementia']
4,40298,100249188,10052853628,Z7830185,3.143184e+09,2017-04-24 00:00:00.0000000,1489471706,MR AVS Snapshot,1,1.433163e+09,...,[' memory'],"[(3762, 4152), (5217, 5610)]",------- mouth nightly. gabapentin (NEURONTIN...,801,12097.0,1007039.0,1007039.0,1,NaN,['memory']


In [203]:
df_not_padded.columns

Index(['index', 'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'buckets', 'NoteCSNID', 'LineNBR',
       'NoteTXT', 'regex_location', 'regex_sent', 'regex_match', 'match_count',
       'pruned_regex_location', 'merged_row_location', 'merged_regex_sent',
       'merged_regex_match', 'padded_merged_row_location',
       'padded_merged_regex_sent', 'padded_char_count', 'note_char_count',
       'PCPID', 'EncounterEpicProviderID', 'pcp_flag', 'SpecialtyDSC',
       'padded_regex_match'],
      dtype='object')

In [206]:
df_not_padded = df_not_padded.drop(columns = ['buckets', 'LineNBR', 'regex_location', 'regex_sent', 'regex_match', 'match_count','pruned_regex_location', 'merged_row_location','merged_regex_match', 'padded_char_count'])

In [207]:
df_not_padded.columns

Index(['index', 'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT',
       'merged_regex_sent', 'padded_merged_row_location',
       'padded_merged_regex_sent', 'note_char_count', 'PCPID',
       'EncounterEpicProviderID', 'pcp_flag', 'SpecialtyDSC',
       'padded_regex_match'],
      dtype='object')

In [208]:
df_not_padded = df_not_padded.drop(columns = ['note_char_count', 'PCPID',
       'EncounterEpicProviderID', 'pcp_flag', 'SpecialtyDSC',])

In [210]:
df_not_padded = df_not_padded.drop(columns = ['padded_merged_row_location'])

In [219]:
df_not_padded = df_not_padded.drop(columns = ['index'])

In [220]:
(df_not_padded.columns)

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT',
       'padded_merged_regex_sent', 'padded_regex_match'],
      dtype='object')

In [216]:
df_not_padded = df_not_padded.drop(columns = ['merged_regex_sent'])

In [221]:
example.columns

Index(['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match'],
      dtype='object')

In [222]:
df_not_padded

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,NoteCSNID,NoteTXT,padded_merged_regex_sent,padded_regex_match
0,100129465,5114654,Z6923710,3.121598e+09,2016-06-20 00:00:00.0000000,1096403210,Progress Notes,1.045850e+09,----------------------------------------------...,"neuralgia, didn't tolerate oxcarbazepin, bacl...",['memory']
1,100335779,10020477146,Z8490045,3.224430e+09,2019-01-16 00:00:00.0000000,2294910337,Progress Notes,2.263433e+09,----------------------------------------------...,nicteric. [Respiratory]: (+) Slight crackles a...,['cognition']
2,100053905,10041789552,Z6458881,3.176475e+09,2017-10-26 00:00:00.0000000,1736629435,Administrative,1.687146e+09,----------------------------------------------...,e. Patient requires physical assistance for pu...,['memory']
3,100067895,62090,Z6522871,3.243889e+09,2019-04-15 00:00:00.0000000,2415533763,Progress Notes,2.385243e+09,----------------------------------------------...,Emotionally abused: Not on file Physically ab...,['dementia']
4,100249188,10052853628,Z7830185,3.143184e+09,2017-04-24 00:00:00.0000000,1489471706,MR AVS Snapshot,1.433163e+09,----------------------------------------------...,------- mouth nightly. gabapentin (NEURONTIN...,['memory']
...,...,...,...,...,...,...,...,...,...,...,...
5004,100014582,10040240250,Z6364707,3.202998e+09,2018-05-29 00:00:00.0000000,2001546644,Consults,1.959757e+09,----------------------------------------------...,cemaker 08/23/2016 • Presence of prosthetic he...,['cerebrovascular']
5005,100011780,10040169418,Z6362262,3.259835e+09,2019-07-11 00:00:00.0000000,2548501334,Consults,2.521082e+09,----------------------------------------------...,"------- , Needs assistance with Mobility Acti...","['cognition', 'memory']"
5006,100000920,204859,Z6352946,3.153583e+09,2017-04-19 00:00:00.0000000,1484554679,Progress Notes,1.428097e+09,----------------------------------------------...,7 for a 30 day supply. Patient's granddaughter...,['memory']
5007,100001150,10027033512,Z6353136,3.121057e+09,2016-06-07 00:00:00.0000000,1068603135,ED Provider Notes,1.019357e+09,----------------------------------------------...,ST Elevations: No Other Finding(s): Non-speci...,['dementia']


In [223]:
df_not_padded.columns = ['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT', 'regex_sent', 'regex_match']

In [224]:
df_not_padded = df_not_padded[['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match']]

In [225]:
df_not_padded

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6923710,5114654,100129465,3.121598e+09,Progress Notes,1096403210,1.045850e+09,2016-06-20 00:00:00.0000000,----------------------------------------------...,"neuralgia, didn't tolerate oxcarbazepin, bacl...",['memory']
1,Z8490045,10020477146,100335779,3.224430e+09,Progress Notes,2294910337,2.263433e+09,2019-01-16 00:00:00.0000000,----------------------------------------------...,nicteric. [Respiratory]: (+) Slight crackles a...,['cognition']
2,Z6458881,10041789552,100053905,3.176475e+09,Administrative,1736629435,1.687146e+09,2017-10-26 00:00:00.0000000,----------------------------------------------...,e. Patient requires physical assistance for pu...,['memory']
3,Z6522871,62090,100067895,3.243889e+09,Progress Notes,2415533763,2.385243e+09,2019-04-15 00:00:00.0000000,----------------------------------------------...,Emotionally abused: Not on file Physically ab...,['dementia']
4,Z7830185,10052853628,100249188,3.143184e+09,MR AVS Snapshot,1489471706,1.433163e+09,2017-04-24 00:00:00.0000000,----------------------------------------------...,------- mouth nightly. gabapentin (NEURONTIN...,['memory']
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6364707,10040240250,100014582,3.202998e+09,Consults,2001546644,1.959757e+09,2018-05-29 00:00:00.0000000,----------------------------------------------...,cemaker 08/23/2016 • Presence of prosthetic he...,['cerebrovascular']
5005,Z6362262,10040169418,100011780,3.259835e+09,Consults,2548501334,2.521082e+09,2019-07-11 00:00:00.0000000,----------------------------------------------...,"------- , Needs assistance with Mobility Acti...","['cognition', 'memory']"
5006,Z6352946,204859,100000920,3.153583e+09,Progress Notes,1484554679,1.428097e+09,2017-04-19 00:00:00.0000000,----------------------------------------------...,7 for a 30 day supply. Patient's granddaughter...,['memory']
5007,Z6353136,10027033512,100001150,3.121057e+09,ED Provider Notes,1068603135,1.019357e+09,2016-06-07 00:00:00.0000000,----------------------------------------------...,ST Elevations: No Other Finding(s): Non-speci...,['dementia']


In [226]:
df_not_padded['regex_sent'] = df_not_padded['regex_sent'].apply(json.dumps)
df_not_padded['regex_match'] = df_not_padded['regex_match'].apply(json.dumps)

In [227]:
merged = df_not_padded #pd.concat([df, df_not_padded])

In [228]:
merged

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6923710,5114654,100129465,3.121598e+09,Progress Notes,1096403210,1.045850e+09,2016-06-20 00:00:00.0000000,----------------------------------------------...,""" neuralgia, didn't tolerate oxcarbazepin, bac...","""['memory']"""
1,Z8490045,10020477146,100335779,3.224430e+09,Progress Notes,2294910337,2.263433e+09,2019-01-16 00:00:00.0000000,----------------------------------------------...,"""nicteric. [Respiratory]: (+) Slight crackles ...","""['cognition']"""
2,Z6458881,10041789552,100053905,3.176475e+09,Administrative,1736629435,1.687146e+09,2017-10-26 00:00:00.0000000,----------------------------------------------...,"""e. Patient requires physical assistance for p...","""['memory']"""
3,Z6522871,62090,100067895,3.243889e+09,Progress Notes,2415533763,2.385243e+09,2019-04-15 00:00:00.0000000,----------------------------------------------...,""" Emotionally abused: Not on file Physically a...","""['dementia']"""
4,Z7830185,10052853628,100249188,3.143184e+09,MR AVS Snapshot,1489471706,1.433163e+09,2017-04-24 00:00:00.0000000,----------------------------------------------...,""" ------- mouth nightly. gabapentin (NEURONTI...","""['memory']"""
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6364707,10040240250,100014582,3.202998e+09,Consults,2001546644,1.959757e+09,2018-05-29 00:00:00.0000000,----------------------------------------------...,"""cemaker 08/23/2016 \u2022 Presence of prosthe...","""['cerebrovascular']"""
5005,Z6362262,10040169418,100011780,3.259835e+09,Consults,2548501334,2.521082e+09,2019-07-11 00:00:00.0000000,----------------------------------------------...,""" ------- , Needs assistance with Mobility Act...","""['cognition', 'memory']"""
5006,Z6352946,204859,100000920,3.153583e+09,Progress Notes,1484554679,1.428097e+09,2017-04-19 00:00:00.0000000,----------------------------------------------...,"""7 for a 30 day supply. Patient's granddaughte...","""['memory']"""
5007,Z6353136,10027033512,100001150,3.121057e+09,ED Provider Notes,1068603135,1.019357e+09,2016-06-07 00:00:00.0000000,----------------------------------------------...,""" ST Elevations: No Other Finding(s): Non-spec...","""['dementia']"""


In [188]:
drop = [100566134,
 101272386,
 103302871,
 103401919,
 104131445,
 104434122,
 104846067,
 105444588,
 106594083,
 106950781,
 107158726,
 107359494,
 110961570,
 111505997,
 100141139,
 100274613,
 100344755,
 100355466,
 100617631,
 100855298,
 101766574,
 101770739,
 101959132,
 102020367,
 102470097,
 103029314,
 103040962,
 103097401,
 103789921,
 104297481,
 104390508,
 104772819,
 105537030,
 105807236,
 108951869,
 111132607,
 111172213]

In [189]:
x = merged
merged_clone = merged

In [190]:
for i in range(len(drop)):
    x = merged_clone[merged_clone["EMPI"] != drop[i]]
    merged_clone = x

In [191]:
merged = merged_clone

In [229]:
merged["ContactDTS"] = pd.to_datetime(merged["ContactDTS"]).dt.strftime('%Y-%m-%d')

In [230]:
merged

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6923710,5114654,100129465,3.121598e+09,Progress Notes,1096403210,1.045850e+09,2016-06-20,----------------------------------------------...,""" neuralgia, didn't tolerate oxcarbazepin, bac...","""['memory']"""
1,Z8490045,10020477146,100335779,3.224430e+09,Progress Notes,2294910337,2.263433e+09,2019-01-16,----------------------------------------------...,"""nicteric. [Respiratory]: (+) Slight crackles ...","""['cognition']"""
2,Z6458881,10041789552,100053905,3.176475e+09,Administrative,1736629435,1.687146e+09,2017-10-26,----------------------------------------------...,"""e. Patient requires physical assistance for p...","""['memory']"""
3,Z6522871,62090,100067895,3.243889e+09,Progress Notes,2415533763,2.385243e+09,2019-04-15,----------------------------------------------...,""" Emotionally abused: Not on file Physically a...","""['dementia']"""
4,Z7830185,10052853628,100249188,3.143184e+09,MR AVS Snapshot,1489471706,1.433163e+09,2017-04-24,----------------------------------------------...,""" ------- mouth nightly. gabapentin (NEURONTI...","""['memory']"""
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6364707,10040240250,100014582,3.202998e+09,Consults,2001546644,1.959757e+09,2018-05-29,----------------------------------------------...,"""cemaker 08/23/2016 \u2022 Presence of prosthe...","""['cerebrovascular']"""
5005,Z6362262,10040169418,100011780,3.259835e+09,Consults,2548501334,2.521082e+09,2019-07-11,----------------------------------------------...,""" ------- , Needs assistance with Mobility Act...","""['cognition', 'memory']"""
5006,Z6352946,204859,100000920,3.153583e+09,Progress Notes,1484554679,1.428097e+09,2017-04-19,----------------------------------------------...,"""7 for a 30 day supply. Patient's granddaughte...","""['memory']"""
5007,Z6353136,10027033512,100001150,3.121057e+09,ED Provider Notes,1068603135,1.019357e+09,2016-06-07,----------------------------------------------...,""" ST Elevations: No Other Finding(s): Non-spec...","""['dementia']"""


In [195]:
merged['regex_sent'] = merged['regex_sent'].apply(json.dumps)
merged['regex_match'] = merged['regex_match'].apply(json.dumps)

In [183]:
merged = merged[:1000]

In [231]:
merged.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_SLAT_import_7_19.csv", index = False)

In [76]:
merged["ContactDTS"] = pd.to_datetime(merged["ContactDTS"]).dt.strftime('%Y-%m-%d')

In [78]:
merged.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\SLAT_dataset_7_14.csv", index = False)

In [66]:
df.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_not_padded_matches.xlsx", index = False)

In [83]:
snippet = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\SLAT_dataset_7_14.csv")

In [84]:
snippet = snippet[:1000]

In [133]:
snippet

,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,10040002098,100000272,3.076070e+09,Progress Notes,471433064,4.532520e+08,2015-06-17,----------------------------------------------...,"""ked at HMS in the department of psychobiology...","""['cerebral']"""
1,10040002098,100000272,3.076214e+09,Progress Notes,513142600,4.924833e+08,2015-07-16,----------------------------------------------...,"""or lymphadenopathy. Chest: Clear to auscultat...","""['memory']"""
2,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...","""['dementia']"""
3,10040002098,100000272,3.322836e+09,Progress Notes,4959788710,4.951007e+09,2020-12-04,----------------------------------------------...,""" ------- ing tylenol really accept for this w...","""['cognition', 'memory', 'moca']"""
4,10040002098,100000272,3.325561e+09,Progress Notes,4570521668,4.558686e+09,2020-09-18,----------------------------------------------...,""" ------- in is related to cancer. Reassuringl...","""['cognition', 'memory']"""
...,...,...,...,...,...,...,...,...,...,...
995,10040097643,100008870,3.188221e+09,Plan of Care,1846564690,1.800569e+09,2018-02-01,----------------------------------------------...,"""elf-care Deficit - Neurological Condition - A...","""['cerebral']"""
996,10040097643,100008870,3.188221e+09,Consults,1846622314,1.800356e+09,2018-02-01,----------------------------------------------...,"""---------------------------------------------...","""['cerebellar']"""
997,10040097643,100008870,3.188221e+09,Consults,1846733081,1.800505e+09,2018-02-01,----------------------------------------------...,""" ------- Deferred- Requiring nursing assista...","""['cerebellar']"""
998,10040097643,100008870,3.188221e+09,Progress Notes,1847368580,1.801464e+09,2018-02-01,----------------------------------------------...,"""---------------------------------------------...","""['cerebellar', 'cerebral']"""


In [85]:
snippet.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\SLAT_snippet.csv", index = False)